In [ ]:
import sympy as sm
import sympy.physics.mechanics as me
import time
import numpy as np
from scipy.integrate import solve_ivp
from scipy.optimize import fsolve, minimize
import matplotlib.pyplot as plt
import matplotlib as mp
from matplotlib import patches
%matplotlib inline
from IPython.display import HTML
mp.rcParams['animation.embed_limit'] = 2**128
from matplotlib import animation

**A chain**

I want to simulate a chain, which I model as a simple open 2D n-link pendulum, where each link is modelled a a thin rod.\
I saw this some time ago as an example of some other modelling library, and I want to see, how this will work with sympy mechanics.\

I noticed the following:
- If I define the rotation of frame $A[i]$ relative to frame $A[i-1]$ the number of operations shoot up: for $n = 12$ the **mass matrix has 25 mio operations**. If I define the rotation of $A[i]$ relative to the inertial frame $O$, the numbers become very reasonable, for $n = 50$, around 40,000 operations.
- Setting up Kane's equations seems to take quite long, given the low numbers of operations in the mass matrix and the force vector. (This is by my experience, I have run very many examples, but certainly not 'scientific'.)
- Also the *lambdification* runs quite long given the small number of operations, again by my experience. With a system, where the mass matrix has 40,000 operations, the force vector has 50,000 operations and the rection force vector has 1,700 operations setting up Kane's equations normally takes a few seconds, here it takes 2 min! 

**Symbols**

- $O$: Reference frame
- $PO$: point fixed in $O$
- $A[i]$: body fixed frame of link $i$ with $0 \leq n < n$
- $Dmc[i]$: center of gravitiy of link $i$
- $P[i]$: point, where frame $A[i]$ joins frame $A[i+1]$
- $l$: length of the pendulum, that is each link has length = $\dfrac{l}{n}$

- $m$: mass of each link
- $iZZ$: moment of inertial of each link around $A[i].z$, relative to $Dmc[i]$
- $reibung$: speed dependent friction in each joint.
- $auxx, auxy, fx, fy$: virtual speeds and reaction forces on point $O$
- $rhs$: 'place holders' for the $rhs = MM^{-1} \cdot force$ to be evaluated numerically later

- $q[i]$: generalized coordinate of frame $A[i]$ relative to the inertial frame $O$.
- $u[i]$: angular speed dto.

In [ ]:
#==================
n = 50
term_info = False
#==================
start = time.time()

m, g, iZZ, l, reibung = sm.symbols('m, g, iZZ, l, reibung')
q   = me.dynamicsymbols(f'q:{n}')
u   = me.dynamicsymbols(f'u:{n}')

auxx, auxy, fx, fy = me.dynamicsymbols('auxx, auxy, fx, fy')
t = me.dynamicsymbols._t

A   = sm.symbols(f'A:{n}', cls=me.ReferenceFrame)
Dmc = sm.symbols(f'Dmc:{n}', cls=me.Point)
P   = sm.symbols(f'P:{n}', cls=me.Point)
rhs = list(sm.symbols(f'rhs:{n}'))

O   = me.ReferenceFrame('O')
PO  = me.Point('PO')
PO.set_vel(O, auxx*O.x + auxy*O.y)

l1 = l/n
l2 = l/(2 * n)

A[0].orient_axis(O, q[0], O.z)
A[0].set_ang_vel(O, u[0] * O.z)

Dmc[0].set_pos(PO, l2*A[0].x)
Dmc[0].v2pt_theory(PO, O, A[0])
P[0].set_pos(PO, l1*A[0].x)
P[0].v2pt_theory(PO, O, A[0])

for i in range(1, n):
    A[i].orient_axis(O, q[i], O.z)         #(A[i-1], q[i], A[i-1].z)
    A[i].set_ang_vel(O, u[i] * O.z)    #(A[i-1], u[i] * A[i-1].z)

    Dmc[i].set_pos(P[i-1], l2*A[i].x)
    Dmc[i].v2pt_theory(P[i-1], O, A[i])
    P[i].set_pos(P[i-1], l1*A[i].x)
    P[i].v2pt_theory(P[i-1], O, A[i])

BODY = []
for i in range(n):
    inertia = me.inertia(A[i], 0., 0., iZZ)
    body    = me.RigidBody('body' + str(i), Dmc[i], A[i], m, (inertia, Dmc[i]))
    BODY.append(body)

FL1    = [(Dmc[i], -m*g*O.y) for i in range(n)] + [(PO, fx*O.x + fy*O.y)]
Torque = [(A[i], - u[i] * reibung * A[i].z) for i in range(n)]
FL     = FL1 + Torque

kd     = [u[i] - q[i].diff(t) for i in range(n)]
aux    = [auxx, auxy]

KM = me.KanesMethod(O, q_ind=q, u_ind=u, kd_eqs=kd, u_auxiliary=aux)
(fr, frstar) = KM.kanes_equations(BODY, FL)

MM = KM.mass_matrix_full
if term_info == True:
    print('MM DS', me.find_dynamicsymbols(MM))
    print('MM free symbols', MM.free_symbols)
    print('MM contains {} operations'.format(sum([MM[i, j].count_ops(visual=False) for i in range(MM.shape[0]) for j in range(MM.shape[1])])), '\n')

force = KM.forcing_full
if term_info == True:
    print('force DS', me.find_dynamicsymbols(force))
    print('force free symbols', force.free_symbols)
    print('force contains {} operations'.format(sum([force[i].count_ops(visual=False) for i in range(force.shape[0])])), '\n')

eingepraegt_dict = {sm.Derivative(i, t): j for i, j in zip(u, rhs)} 
eingepraegt      = KM.auxiliary_eqs.subs(eingepraegt_dict)
if term_info == True:
    print('eingepraegt DS', me.find_dynamicsymbols(eingepraegt))
    print('eingepraegt free symbols', eingepraegt.free_symbols)
    print('eingepraegt has {} operations'.format(sum([eingepraegt[i].count_ops(visual=False) for i in range(len(eingepraegt))])), '\n')
print(f'it took {time.time() - start:.3f} sec to set up Kanes equations for {n} links')

Functions for the kinetic and the potential energies.\
Always useful to detect mistakes.\
orte is needed for the animation only.

In [ ]:
kin_energie = sum([koerper.kinetic_energy(O) for koerper in BODY]).subs({i: 0. for i in aux})
pot_energie = sum([m*g*me.dot(koerper.pos_from(PO), O.y) for koerper in Dmc])

orte        = [[me.dot(P[i].pos_from(PO), uv) for uv in (O.x, O.y)] for i in range(n) ]

Lambdification.\
As mentioned above, it takes quite long to run.

In [ ]:
start3 = time.time()
qL = q + u
pL = [m, g, l, iZZ, reibung]
F  = [fx, fy]

MM_lam          = sm.lambdify(qL + pL, MM, cse=True)
force_lam       = sm.lambdify(qL + pL, force, cse=True)
eingepraegt_lam = sm.lambdify(F + qL + pL + rhs, eingepraegt, cse=True)

kin_lam = sm.lambdify(qL + pL, kin_energie, cse=True)
pot_lam = sm.lambdify(qL + pL, pot_energie, cse=True)
ort_lam = sm.lambdify(qL + pL, orte, cse=True)

print(f'it took {time.time()-start3:.3f} sec to do the lambdification')

**Numerical Integration**

Nothing special here.\
method='Radau' in solve_ivp gives a more constant total energy if $reibung = 0.$

In [ ]:
start2 = time.time()
#==========================
# Input variables
#==========================
m1       = 1.
g1       = 9.8
l1       = 20
reibung1 = 0.

q1      = [3.*np.pi/2. + np.pi * i / n for i in range(1, n+1)]
u1      = [0. for _ in range(n)]

intervall = 10.
punkte    = 50
#==========================

schritte = int(intervall * punkte)
times    = np.linspace(0., intervall, schritte)
t_span   = (0., intervall)

iZZ1 = 1./12. * m1 * (l1/n)**2        # from the internet

pL_vals = [m1, g1, l1, iZZ1, reibung1]
y0      =[*q1, *u1]


def gradient(t, y, args):
    sol = np.linalg.solve(MM_lam(*y, *args), force_lam(*y, *args))
    return np.array(sol).T[0]
        
resultat1 = solve_ivp(gradient, t_span, y0, t_eval = times, args=(pL_vals,), method='Radau')
#                , atol=1.e-3, rtol=1.e-3)

resultat = resultat1.y.T
print('resultat shape', resultat.shape, '\n')
event_dict = {-1: 'Integration failed', 0: 'Integration finished successfully', 1: 'some termination event'}
print(event_dict[resultat1.status], ', message is:', resultat1.message, '\n')

print("To numerically integrate an intervall of {} sec the routine cycled {} times and it took {:.3f} sec "
      .format(intervall, resultat1.nfev, time.time() - start2), '\n')

Plot the energies.

In [ ]:
kin_np   = np.empty(schritte)
pot_np   = np.empty(schritte)
total_np = np.empty(schritte)

for i in range(schritte):
    kin_np[i] = kin_lam(*[resultat[i, j] for j in range(resultat.shape[1])], *pL_vals)
    pot_np[i] = pot_lam(*[resultat[i, j] for j in range(resultat.shape[1])], *pL_vals)
    total_np[i] = kin_np[i] + pot_np[i]

if reibung1 == 0.:
    max_total = np.max(np.abs(total_np))
    min_total = np.min(np.abs(total_np))
    delta = max_total - min_total
    print(f'max deviation of total energy from zero is {delta/max_total * 100:.3e} % of max. total energy')
fig, ax = plt.subplots(figsize=(10, 5))
for i, j in zip((kin_np, pot_np, total_np), ('kinetic energy', 'potential energy', 'total energy')):
    ax.plot(times, i, label=j)
ax.set_title("Energies of the system")
ax.set_xlabel('time (sec)')
ax.set_ylabel('energy (Nm)')
ax.legend();
    

**Reaction force on $PO$**

First I solve the equation $rhs = MM^{-1} \cdot force$ numerically.\
Then I solve $eingepraegt = 0$ for $fx, fy$ numerically.\
I iterate over fsolve twice, this sometimes improves the accuracy.\
I iterate over the whole process twice: Sometimes there is a message that convergence is slow. If I run it again, usually this message does not come again. I do not know, why.


In [ ]:
# RHS calculated numerically, too large to do it symbolically. Needed for reaction forces.

# this is only needed because I basically copied it from some other program, where resultat2, etc 
# was different from resultat.
resultat2 = resultat
schritte2 = schritte
times2    = times

RHS1 = np.empty((schritte2, resultat2.shape[1]))
for i in range(schritte2):
    RHS1[i, :] = np.linalg.solve(MM_lam(*[resultat2[i, j]for j in range(resultat2.shape[1])], *pL_vals), 
        force_lam(*[resultat2[i, j] for j in range(resultat2.shape[1])], 
        *pL_vals)).reshape(resultat2.shape[1])
print('RHS1 shape', RHS1.shape)

#calculate implied forces numerically
def func (x, *args):
# just serves to 'modify' the arguments for fsolve.
    return eingepraegt_lam(*x, *args).reshape(2)

for _ in range(2):
    kraftx = np.empty(schritte2)
    krafty = np.empty(schritte2)
    x0 = tuple((1., 1.))   # initial guess

    for i in range(schritte2):
        for _ in range(2):
            y0 = [resultat2[i, j] for j in range(resultat2.shape[1])]
            rhs = [RHS1[i, j] for j in range(n, 2*n)]
            args = tuple(y0 + pL_vals + rhs)
            AAA = fsolve(func, x0, args=args).reshape(2)
            x0 = tuple(AAA)      # improved guess. Should speed up convergence of fsolve
        kraftx[i] = AAA[0]
        krafty[i] = AAA[1]

fig, ax = plt.subplots(figsize=(10, 5))
plt.plot(times2, kraftx, label='X force')
plt.plot(times2, krafty, label='Y force')
ax.set_title('Reaction Forces on suspension point PO')
ax.set_xlabel('time (sec)')
ax.set_ylabel('force (N)')
plt.legend();

**Animation**

In [ ]:
# get Carthesian coordinates
x_coords = np.empty((len(times), n+1))
y_coords = np.empty((len(times), n+1))

for j in range(schritte):
    x_coords[j] = [0.] + [ort_lam(*[resultat[j, k] for k in range(resultat.shape[1])], *pL_vals)[k11][0] 
                         for k11 in range(n) ]
    y_coords[j] = [0.] + [ort_lam(*[resultat[j, k] for k in range(resultat.shape[1])], *pL_vals)[k11][1] 
                         for k11 in range(n)]

max_x = max([abs(x_coords[i, j])for i in range(schritte) for j in range(n+1)])
max_y = max([abs(y_coords[i, j])for i in range(schritte) for j in range(n+1)])
max_xy = max(max_x, max_y)

fig, ax = plt.subplots(figsize=(7, 7))
ax.axis('on')
ax.set(xlim=(-max_xy-1., max_xy+1.), ylim=(-max_xy-1., max_xy+1.))
ax.plot(0., 0.4, marker='v', markersize=15, color='red')

Lines = []
for i in range(n):     # sets the points. Red points are fixed
    line, = ax.plot([], [], 'o-', lw=0.5, color='blue', markersize=0.)
    Lines.append(line)

line, = ax.plot([], [], 'o-', lw=0.5, color='blue', markersize=0) #connects the points
Lines.append(line)
   
def animate_pendulum(times, x, y):    
    def animate(i):
        ax.set_title('Running time is {:.2f} sec'.format(i / schritte * intervall), fontsize=12)
        for j in range(n+1):
            Lines[j].set_data(x[i, j], y[i, j])
        Lines[-1].set_data(x[i], y[i])
        return Lines

    anim = animation.FuncAnimation(fig, animate, frames=len(times),
                                   interval=1000*times.max() / len(times),
                                   blit=True) #, init_func=init)
    plt.close(fig)
    return anim

anim = animate_pendulum(times, x_coords, y_coords)
print('to run the program before HTML(..) took {:.3f} sec'.format(time.time() - start))
HTML(anim.to_jshtml())